 ___Sentiment Analysis on tomato Reviews using Random and TF-IDF Approach.___

__Feature Extraction__: TF-IDF (term frequency-inverse document frequency)

__Classification__: SVM, Logistic Regression 

In this notebook, Sentiment Analysis is performed on dataset.

Import the Required Packages:

In [ ]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.2 MB/s eta 0:00:00


In [ ]:
import wandb
wandb.login(key="aec6fef7ba56ee445129472eb583718b8e529934")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import gzip
import pickle
import pandas as pd
import numpy as np
import nltk
import re
import random
from tqdm import tqdm
import json
import wandb
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/mlops/mlops_part_2_progect/reviews

/content/drive/MyDrive/mlops/mlops_part_2_progect/reviews


In [ ]:

with open('train.pkl', 'rb') as f:
    train_set = pickle.load(f)
    
with open('test.pkl', 'rb') as f:
    test_set = pickle.load(f)
        
 

In [ ]:
train = pd.DataFrame.from_dict(train_set)
test =  pd.DataFrame.from_dict(test_set)

In [ ]:
train = train.dropna()
test = test.dropna()

In [ ]:
X_train = train["x"]
X_test = test["x"]
Y_train = train["y"]
Y_test = test["y"]


# Random
 <u><i>motivation</i></u>: We want to know what random (chance) performance looks like. All of our efforts should be well above this baseline.

In [ ]:
# Generate random predictions

label = ["Fresh","Rotten"]
y_pred = []
for i in range(len(Y_test)):
    y_pred.append(random.choice(label))

from sklearn.metrics import classification_report
metrics=classification_report(Y_test, y_pred,output_dict=True)


In [ ]:
from sklearn.metrics import classification_report
metrics_random=classification_report(Y_test, y_pred,output_dict=True)
performance = {"accuracy": metrics_random["accuracy"],
               "precision": metrics_random["weighted avg"]['precision'],
               "recall": metrics_random["weighted avg"]['recall'],
               "f1": metrics_random["weighted avg"]['f1-score']}

In [ ]:
print (json.dumps(performance, indent=2))

{
  "accuracy": 0.49913604180714766,
  "precision": 0.5385056279670113,
  "recall": 0.49913604180714766,
  "f1": 0.5091301023977443
}


By Using the __random__ classifier we got an accuracy of 49.91%

__weighted random__

We made the assumption that there is an equal probability for every class. Let's use the train split to figure out what the true probability is.

In [ ]:
# Class frequencies
import collections
collections.Counter(Y_test)

Counter({'Fresh': 60733, 'Rotten': 34179})

In [ ]:
p=[]
for i in label:
     p.append(collections.Counter(Y_test)[i]/len(Y_test))

In [ ]:
y_pred = []
for i in range(len(Y_test)):
    y_pred.append(random.choices(label, weights=p ,k=1) )

In [ ]:
metrics_weighted_random = classification_report(Y_test, y_pred,output_dict=True)
performance_weighted_random = {"accuracy": metrics_weighted_random["accuracy"],
               "precision": metrics_weighted_random["weighted avg"]['precision'],
               "recall": metrics_weighted_random["weighted avg"]['recall'],
               "f1": metrics_weighted_random["weighted avg"]['f1-score']}
print (json.dumps(performance_weighted_random, indent=2))


{
  "accuracy": 0.5405322825354012,
  "precision": 0.5410753536142617,
  "recall": 0.5405322825354012,
  "f1": 0.5408019102394513
}


By Using the __weighted random__ classifier we got an accuracy of 54.05%

# Feature Extraction
__TF-IDF__: It is a method of extracting the features from the text data. TF stands for Term Frequency and IDF stands for Inverse Document Frequency.

__Term Frequency__: Number of times word occurs in a review. For an example consider 2 reviews where w1,w2.. represents the words in both reviews and table defines the frequency of words in the particular review.

IDF is computed as :

    idf(t) = log [ n / df(t) ] + 1
         = log [ number of documents / number of documents containing the term]+1
         
         
If smooth_idf=True.
        Smooth-IDF = log [ n / df(t) +1 ] + 1         


TF-IDF is implemented using sklearn : [link](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)




__Using TF*IDF Vectorizer__

In [ ]:
print("TFIDF Vectorizer……")
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)

TFIDF Vectorizer……


__SVM__

Implementing SVM with sklearn for classification

In [ ]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0,verbose=True)

In [ ]:
clf.fit(tf_x_train,Y_train)

[LibLinear]

LinearSVC(random_state=0, verbose=True)

Predicting the Test data

In [ ]:
y_test_pred=clf.predict(tf_x_test)

In [ ]:
metrics_SVM=classification_report(Y_test, y_test_pred,output_dict=True)

In [ ]:
performance_SVM = {"accuracy": metrics_SVM["accuracy"],
               "precision": metrics_SVM["weighted avg"]['precision'],
               "recall": metrics_SVM["weighted avg"]['recall'],
               "f1": metrics_SVM["weighted avg"]['f1-score']}
print (json.dumps(performance_SVM, indent=2))

{
  "accuracy": 0.808780765340526,
  "precision": 0.8062256099340317,
  "recall": 0.808780765340526,
  "f1": 0.8064078913039544
}


__By Using the SVM classifier we got an accuracy of 80.87%__









__Logistic Regression__

Logistic regression is implemented using sklearn


In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=1000,solver="saga")

Fit the Training data to the model

In [ ]:
clf.fit(tf_x_train,Y_train)

LogisticRegression(max_iter=1000, solver='saga')

Predicting the test data

In [ ]:
y_test_pred_Logistic_Regression=clf.predict(tf_x_test)

Analyzing the Report

In [ ]:
metrics_Logistic_Regression = classification_report(Y_test, y_test_pred_Logistic_Regression,output_dict=True)

In [ ]:
performance_Logistic_Regression = {"accuracy": metrics_Logistic_Regression["accuracy"],
               "precision": metrics_Logistic_Regression["weighted avg"]['precision'],
               "recall": metrics_Logistic_Regression["weighted avg"]['recall'],
               "f1": metrics_Logistic_Regression["weighted avg"]['f1-score']}
print (json.dumps(performance_Logistic_Regression, indent=2))

{
  "accuracy": 0.8127528658125421,
  "precision": 0.8102321318158092,
  "recall": 0.8127528658125421,
  "f1": 0.809611957399829
}


__By Using the LR classifier we got an accuracy of 81.27%__

In [ ]:
# Initialize wandb project
wandb.init(project="svm-project")



# Log the metrics and models to wandb
wandb.log({

            "accuracy_weighted_random": metrics_weighted_random["accuracy"],
           "precision_weighted_random": metrics_weighted_random["weighted avg"]['precision'],
           "recall_weighted_random": metrics_weighted_random["weighted avg"]['recall'],
           "f1_weighted_random": metrics_weighted_random["weighted avg"]['f1-score']
            ,

           "accuracy_svm": metrics_SVM["accuracy"],
           "precision_svm": metrics_SVM["weighted avg"]['precision'],
           "recall_svm": metrics_SVM["weighted avg"]['recall'],
           "f1_svm": metrics_SVM["weighted avg"]['f1-score']
           ,
   
           "accuracy_Logistic_Regression": metrics_Logistic_Regression["accuracy"],
           "precision_Logistic_Regression": metrics_Logistic_Regression["weighted avg"]['precision'],
           "recall_Logistic_Regression": metrics_Logistic_Regression["weighted avg"]['recall'],
           "f1_Logistic_Regression": metrics_Logistic_Regression["weighted avg"]['f1-score']
           })

wandb.sklearn.plot_confusion_matrix(Y_test, y_test_pred, label)

wandb.sklearn.plot_confusion_matrix(Y_test, y_test_pred_Logistic_Regression, label)

wandb: Currently logged in as: sentimovie. Use `wandb login --relogin` to force relogin
